In [1]:
use mavenfuzzyfactory;

Commands completed successfully

Total execution time: 00:00:00.003

**Subject: Expanded Channel Portfolio**

<span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">With gsearch doing well and the site performing better, we&nbsp;</span> <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">launched a second paid search channel, bsearch, around&nbsp;</span> <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">August 22.</span>

Can you pull weekly trended session volume since then and <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">compare to search nonbrand so I can get a sense for how&nbsp;</span> <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">important this will be for the business?</span>

In [30]:
select min(date(website_sessions.created_at)) as week_start_date,
count(DISTINCT case when utm_source = 'gsearch' then website_sessions.website_session_id else null end) as gsearch_sessions,
count(DISTINCT case when utm_source = 'bsearch' then website_sessions.website_session_id else null end) as bsearch_sessions
from website_sessions
where website_sessions.created_at > '2012-08-22'
and website_sessions.created_at < '2012-11-29'
and utm_campaign = 'nonbrand'
GROUP BY YEARWEEK(date(website_sessions.created_at))


(15 row(s) affected)

Total execution time: 00:00:01.246

week_start_date,gsearch_sessions,bsearch_sessions
2012-08-22,590,197
2012-08-26,1056,343
2012-09-02,925,290
2012-09-09,951,329
2012-09-16,1151,365
2012-09-23,1050,321
2012-09-30,999,316
2012-10-07,1002,330
2012-10-14,1257,420
2012-10-21,1302,431


**Subject: Comparing Our Channels**

I'd like to learn more about the bsearch nonbrand campaign. <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">Could you please pull the percentage of traffic coming on&nbsp;</span> <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">Mobile, and compare that to search?&nbsp;</span> <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">Aggregate data since August 22nd is great, no&nbsp;</span> <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">need to show trending at this point.</span>

In [33]:
select * from website_sessions limit 5

(5 row(s) affected)

Total execution time: 00:00:01.008

website_session_id,created_at,user_id,is_repeat_session,utm_source,utm_campaign,utm_content,device_type,http_referer
1,2012-03-19 08:04:16,1,0,gsearch,nonbrand,g_ad_1,mobile,https://www.gsearch.com
2,2012-03-19 08:16:49,2,0,gsearch,nonbrand,g_ad_1,desktop,https://www.gsearch.com
3,2012-03-19 08:26:55,3,0,gsearch,nonbrand,g_ad_1,desktop,https://www.gsearch.com
4,2012-03-19 08:37:33,4,0,gsearch,nonbrand,g_ad_1,desktop,https://www.gsearch.com
5,2012-03-19 09:00:55,5,0,gsearch,nonbrand,g_ad_1,mobile,https://www.gsearch.com


In [34]:
select 
utm_source,
count(DISTINCT website_session_id) as sessions,
count(DISTINCT case when device_type = 'mobile' then website_session_id else null end) as mobile_sessions,
count(DISTINCT case when device_type = 'mobile' then website_session_id else null end)/count(DISTINCT website_session_id) as pct_mobile
from website_sessions
where created_at > '2012-08-22'
and created_at < '2012-11-29'
and utm_campaign = 'nonbrand'
GROUP BY utm_source


(2 row(s) affected)

Total execution time: 00:00:01.271

utm_source,sessions,mobile_sessions,pct_mobile
bsearch,6460,558,0.0864
gsearch,19884,4867,0.2448


**Subject: Multi-Channel Bidding**

<span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">Could you pull nonbrand conversion rates&nbsp;</span> <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">from session to order for gsearch and bsearch, and slice the&nbsp;</span> <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">data by device type?&nbsp;</span> <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">Please analyze data from August 22 to September 18; we&nbsp;</span> <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">ran a special pre-holiday campaign for gsearch starting on&nbsp;</span> <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">September 19th, so the data after that isn't fair game.</span>

In [36]:
select * from orders limit 5

(5 row(s) affected)

Total execution time: 00:00:01.006

order_id,created_at,website_session_id,user_id,primary_product_id,items_purchased,price_usd,cogs_usd
1,2012-03-19 10:42:46,20,20,1,1,49.99,19.49
2,2012-03-19 19:27:37,104,104,1,1,49.99,19.49
3,2012-03-20 06:44:45,147,147,1,1,49.99,19.49
4,2012-03-20 09:41:45,160,160,1,1,49.99,19.49
5,2012-03-20 11:28:15,177,177,1,1,49.99,19.49


In [39]:
select 
device_type,
utm_source,
count(DISTINCT website_sessions.website_session_id) as sessions,
count(DISTINCT order_id) as orders,
count(DISTINCT order_id)/count(DISTINCT website_sessions.website_session_id) as conv_rate
from website_sessions
left join orders
on website_sessions.website_session_id = orders.website_session_id
where website_sessions.created_at > '2012-08-22'
and website_sessions.created_at < '2012-09-19'
and utm_campaign = 'nonbrand'
GROUP BY 1, 2

(4 row(s) affected)

Total execution time: 00:00:01.250

device_type,utm_source,sessions,orders,conv_rate
desktop,bsearch,1162,44,0.0379
desktop,gsearch,3011,136,0.0452
mobile,bsearch,130,1,0.0077
mobile,gsearch,1015,13,0.0128


**Subject: Impact of Bid Changes**

Based on your last analysis, we bid down bsearch nonbrand on <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">December 2nd.&nbsp;</span>  <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">Can you pull weekly session volume for search and search&nbsp;</span>  <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">nonbrand, broken down by device, since November 4th?&nbsp;</span>  <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">If you can include a comparison metric to show search as a&nbsp;</span>  <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">percent of gsearch for each device, that would be great too.</span>

In [42]:
select * from website_sessions limit 5

(5 row(s) affected)

Total execution time: 00:00:01.008

website_session_id,created_at,user_id,is_repeat_session,utm_source,utm_campaign,utm_content,device_type,http_referer
1,2012-03-19 08:04:16,1,0,gsearch,nonbrand,g_ad_1,mobile,https://www.gsearch.com
2,2012-03-19 08:16:49,2,0,gsearch,nonbrand,g_ad_1,desktop,https://www.gsearch.com
3,2012-03-19 08:26:55,3,0,gsearch,nonbrand,g_ad_1,desktop,https://www.gsearch.com
4,2012-03-19 08:37:33,4,0,gsearch,nonbrand,g_ad_1,desktop,https://www.gsearch.com
5,2012-03-19 09:00:55,5,0,gsearch,nonbrand,g_ad_1,mobile,https://www.gsearch.com


In [46]:
select 
min(date(created_at)),
count(DISTINCT case when device_type = 'desktop' and utm_source = 'gsearch' then website_session_id else null end) as g_dtop_sessions,
count(DISTINCT case when device_type = 'desktop' and utm_source = 'bsearch' then website_session_id else null end) as b_dtop_sessions,
count(DISTINCT case when device_type = 'desktop' and utm_source = 'bsearch' then website_session_id else null end)/
count(DISTINCT case when device_type = 'desktop' and utm_source = 'gsearch' then website_session_id else null end) as b_pct_of_d_dtop,
count(DISTINCT case when device_type = 'mobile' and utm_source = 'gsearch' then website_session_id else null end) as g_mob_sessions,
count(DISTINCT case when device_type = 'mobile' and utm_source = 'bsearch' then website_session_id else null end) as b_mob_sessions,
count(DISTINCT case when device_type = 'mobile' and utm_source = 'bsearch' then website_session_id else null end)/
count(DISTINCT case when device_type = 'mobile' and utm_source = 'gsearch' then website_session_id else null end) as b_pct_of_d_mob
from website_sessions
where created_at > '2012-11-04'
and created_at < '2012-12-23'
and utm_campaign = 'nonbrand'
GROUP BY yearweek(date(created_at))

(7 row(s) affected)

Total execution time: 00:00:01.329

min(date(created_at)),g_dtop_sessions,b_dtop_sessions,b_pct_of_d_dtop,g_mob_sessions,b_mob_sessions,b_pct_of_d_mob
2012-11-04,1027,400,0.3895,323,29,0.0898
2012-11-11,956,401,0.4195,290,37,0.1276
2012-11-18,2655,1008,0.3797,853,85,0.0996
2012-11-25,2058,843,0.4096,692,62,0.0896
2012-12-02,1326,517,0.3899,396,31,0.0783
2012-12-09,1277,293,0.2294,424,46,0.1085
2012-12-16,1370,383,0.2796,412,49,0.1189
